In [23]:
import os, math, traceback, glob
from collections import defaultdict
import polars as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import time

# ------------------------
# User config
# ------------------------
DATA_ROOT = r"C:\Users\Alqama\QuantX_Trading_Project\Data\1 Min Data\1 Min Data\OHLC"

ALL_TICKERS = [
    "AAPL","MSFT","AMZN","GOOG","META","NVDA",
    "JPM","BAC","GS","MS","WFC",
    "JNJ","UNH","PFE","MRK","ABT",
    "XOM","CVX","CAT","GE","MMM",
    "PG","KO","WMT","MCD","NKE","PEP","HD","DIS",
    "V","MA","ZTS"
]

RUN_MODE = "FULL_YEAR"   # JAN / FEB / JAN_FEB / JAN_JUN / JUL_SEP / OCT_DEC / FULL_YEAR

# ------------------------
# Strategy params (unchanged)
# ------------------------
DAILY_TREND_WINDOW    = 5
INTRADAY_LOOKBACK     = 15
ATR_WINDOW            = 14

Z_THRESHOLD           = 0.65
CONFIRM_BARS          = 0
VOLUME_MIN_FACTOR     = 0.35

RISK_PER_TRADE        = 0.035
MAX_POSITION_FRACTION = 0.05
MAX_GROSS_EXPOSURE    = 2.2
MAX_OPEN_POSITIONS_TOTAL = 14

STOP_LOSS_PCT         = 0.022
TAKE_PROFIT_PCT       = 0.10

TRANSACTION_COST_PCT  = 0.0002
SLIPPAGE_PCT          = 0.0005
MINUTES_PER_DAY       = 390
SKIP_FIRST_MINUTES    = 3
SKIP_LAST_MINUTES     = 5

INITIAL_CAPITAL       = 1_000_000.0

ROLL_STD_FLOOR = 1e-4
VOL_FLOOR = 1e-4
ATR_FLOOR = 1e-4

# Output paths & behaviour
OUT_DIR = "./quantx_reports"
os.makedirs(OUT_DIR, exist_ok=True)
GENERATE_PDF = True
MAX_TICKER_CHARTS = 32            # max per-ticker images
MAX_TRADES_PER_TICKER_ZOOM = 0    # 0 = disabled (avoid 322 extra charts)

# ------------------------
# RUN_MODE -> dates
# ------------------------
if RUN_MODE == "JAN":
    START_DATE, END_DATE = "2024-01-02", "2024-01-31"
elif RUN_MODE == "FEB":
    START_DATE, END_DATE = "2024-02-01", "2024-02-29"
elif RUN_MODE == "JAN_FEB":
    START_DATE, END_DATE = "2024-01-02", "2024-02-29"
elif RUN_MODE == "JAN_JUN":
    START_DATE, END_DATE = "2024-01-02", "2024-06-28"
elif RUN_MODE == "JUL_SEP":
    START_DATE, END_DATE = "2024-07-01", "2024-09-30"
elif RUN_MODE == "OCT_DEC":
    START_DATE, END_DATE = "2024-10-01", "2024-12-31"
else:
    START_DATE, END_DATE = "2024-01-02", "2024-12-31"

TICKERS = ALL_TICKERS

print(f"QuantX FINAL Backtest | RUN_MODE={RUN_MODE}")
print(f"Running {len(TICKERS)} tickers: {TICKERS}")
print(f"Date range: {START_DATE} → {END_DATE}\n")

# ------------------------
# IO helper (robust & cached)
# ------------------------
_day_cache = {}
def _find_file_for_day(ticker, date_str):
    """
    Look inside DATA_ROOT/ticker for any file containing date_str in its filename and return full path.
    Accepts parquet, csv, pkl. Returns None if nothing found.
    """
    folder = os.path.join(DATA_ROOT, ticker)
    if not os.path.isdir(folder):
        return None
    # search for common extensions
    for ext in ("*.parquet", "*.parq", "*.csv", "*.pkl"):
        for fn in glob.glob(os.path.join(folder, f"*{date_str}*{ext.replace('*','')}")):
            return fn
    # fallback: any file with date_str substring
    for fn in os.listdir(folder):
        if date_str in fn:
            return os.path.join(folder, fn)
    return None

def load_minute_parquet_for_day(ticker, date_str):
    """
    Return pandas DataFrame with columns: timestamp, open, high, low, close, volume, ms_of_day
    or None if not available.
    """
    key = (ticker, date_str)
    if key in _day_cache:
        return _day_cache[key]

    candidate = _find_file_for_day(ticker, date_str)
    if candidate is None:
        _day_cache[key] = None
        return None
    try:
        if candidate.endswith((".parquet",".parq")):
            df_pl = pl.read_parquet(candidate)
            df = df_pl.to_pandas()
        elif candidate.endswith(".csv"):
            df = pd.read_csv(candidate)
        elif candidate.endswith(".pkl"):
            df = pd.read_pickle(candidate)
        else:
            # try read parquet/csv heuristics
            try:
                df_pl = pl.read_parquet(candidate)
                df = df_pl.to_pandas()
            except Exception:
                df = pd.read_csv(candidate)
    except Exception:
        _day_cache[key] = None
        return None

    # normalize columns and construct timestamp (support a couple of naming conventions)
    if 'date' not in df.columns or 'ms_of_day' not in df.columns:
        # try to infer
        if 'timestamp' in df.columns:
            # assume timestamp is epoch ms or ISO string
            try:
                df['timestamp'] = pd.to_datetime(df['timestamp'])
            except Exception:
                pass
        # if we don't have required columns, give up
    # If 'date' exists and 'ms_of_day' exists -> create timestamp
    if 'date' in df.columns and 'ms_of_day' in df.columns:
        df['date_dt'] = pd.to_datetime(df['date'].astype(str), format="%Y%m%d", errors='coerce')
        df['timestamp'] = df['date_dt'] + pd.to_timedelta(df['ms_of_day'], unit='ms')
    # else try parse 'timestamp' column (already)
    if 'timestamp' not in df.columns:
        _day_cache[key] = None
        return None

    # restrict to market hours if we can
    try:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.loc[
            (df['timestamp'].dt.time >= time(9,30)) &
            (df['timestamp'].dt.time <= time(16,0))
        ]
    except Exception:
        pass

    # standardize numeric columns existence
    for c in ['open','high','low','close','volume','ms_of_day']:
        if c not in df.columns:
            df[c] = np.nan

    df = df.sort_values('timestamp').reset_index(drop=True)
    if df.empty:
        _day_cache[key] = None
        return None
    _day_cache[key] = df[['timestamp','open','high','low','close','volume','ms_of_day']].copy()
    return _day_cache[key]

# ------------------------
# Feature calculators
# ------------------------
def compute_daily_trend(ticker, dates):
    closes = []
    for d in dates:
        df = load_minute_parquet_for_day(ticker, d.strftime("%Y%m%d"))
        closes.append(np.nan if df is None or df.empty else df['close'].iloc[-1])
    s = pd.Series(closes, index=dates)
    trend = s.rolling(window=DAILY_TREND_WINDOW, min_periods=DAILY_TREND_WINDOW).mean()
    return s, trend

def compute_intraday_indicators(df_minute):
    if df_minute is None or df_minute.empty:
        return None
    df = df_minute.copy()
    df['ret'] = df['close'].pct_change().fillna(0)
    df['roll_mean'] = df['close'].rolling(INTRADAY_LOOKBACK, min_periods=INTRADAY_LOOKBACK).mean()
    df['roll_std'] = df['close'].rolling(INTRADAY_LOOKBACK, min_periods=INTRADAY_LOOKBACK).std(ddof=0)
    df['roll_std'] = df['roll_std'].replace(0, np.nan).ffill().bfill().fillna(ROLL_STD_FLOOR).abs()
    df.loc[df['roll_std'] < ROLL_STD_FLOOR, 'roll_std'] = ROLL_STD_FLOOR
    df['z'] = (df['close'] - df['roll_mean']) / df['roll_std']
    df['vol15'] = df['volume'].rolling(window=15, min_periods=1).mean()
    high, low, close = df['high'], df['low'], df['close']
    prev_close = close.shift(1).fillna(close)
    tr = pd.concat([
        (high - low).abs(),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)
    df['atr'] = tr.rolling(window=ATR_WINDOW, min_periods=1).mean().fillna(ATR_FLOOR)
    df['volatility'] = (df['atr'] / df['close']).replace([np.inf, -np.inf], np.nan)
    df['volatility'] = df['volatility'].ffill().bfill().fillna(VOL_FLOOR).abs()
    df.loc[df['volatility'] < VOL_FLOOR, 'volatility'] = VOL_FLOOR
    return df

# ------------------------
# Backtest core (sequential)
# ------------------------
def run_backtest(tickers, start_date, end_date, stop_on_negative_cash=False):
    dates = pd.bdate_range(start=start_date, end=end_date)
    daily_closes, daily_trends = {}, {}
    for t in tickers:
        s, trend = compute_daily_trend(t, dates)
        daily_closes[t], daily_trends[t] = s, trend

    cash = float(INITIAL_CAPITAL)
    positions = {t: {'shares':0, 'entry_price':0.0, 'entry_time':None, 'z':np.nan} for t in tickers}
    portfolio_history = [(pd.to_datetime(start_date), cash)]
    trades = []
    stats = defaultdict(int)

    for current_date in dates:
        date_str = current_date.strftime("%Y%m%d")
        print(f"Processing {date_str}")
        try:
            intraday = {t: compute_intraday_indicators(load_minute_parquet_for_day(t, date_str)) for t in tickers}
            if all(v is None for v in intraday.values()):
                portfolio_history.append((pd.to_datetime(current_date) + pd.Timedelta(hours=16), cash))
                continue

            entered_today = {t: False for t in tickers}
            max_len = max((len(df) for df in intraday.values() if df is not None), default=0)
            start_i = SKIP_FIRST_MINUTES
            end_i = max_len - SKIP_LAST_MINUTES - 1
            if end_i <= start_i:
                portfolio_history.append((pd.to_datetime(current_date) + pd.Timedelta(hours=16), cash))
                continue

            for i in range(start_i, end_i):
                # mark-to-market price mapping
                current_prices = {}
                for t, df in intraday.items():
                    if df is None or i+1 >= len(df): continue
                    p = df['open'].iloc[i+1]
                    if np.isfinite(p) and p > 0: current_prices[t] = p

                total_value = cash + sum(pos['shares'] * current_prices.get(sym, pos['entry_price'])
                                         for sym,pos in positions.items() if pos['shares'] != 0)
                gross_exposure = sum(abs(pos['shares']) * current_prices.get(sym, pos['entry_price'])
                                     for sym,pos in positions.items() if pos['shares'] != 0)

                open_positions_count = sum(1 for p in positions.values() if p['shares'] != 0)
                for t, df in intraday.items():
                    if df is None or i+1 >= len(df):
                        stats['missing_data'] += 1
                        continue

                    exec_price = df['open'].iloc[i+1]
                    if not (np.isfinite(exec_price) and exec_price > 0):
                        stats['price_fail'] += 1
                        continue

                    pos = positions[t]

                    # EXIT first (intraday)
                    if pos['shares'] != 0:
                        pnl_pct = (exec_price - pos['entry_price']) / (pos['entry_price'] if pos['entry_price']!=0 else 1e-8)
                        if (pnl_pct <= -STOP_LOSS_PCT) or (pnl_pct >= TAKE_PROFIT_PCT):
                            proceeds = pos['shares'] * exec_price * (1.0 - TRANSACTION_COST_PCT - SLIPPAGE_PCT)
                            cash += proceeds
                            trades.append({
                                'symbol': t,
                                'entry_price': pos['entry_price'],
                                'exit_price': exec_price,
                                'entry_time': pos.get('entry_time'),
                                'exit_time': df['timestamp'].iloc[i+1],
                                'shares': pos['shares'],
                                'pnl': (exec_price - pos['entry_price']) * pos['shares'],
                                'pnl_pct': pnl_pct,
                                'z_score': pos.get('z', np.nan)
                            })
                            positions[t] = {'shares':0,'entry_price':0.0,'entry_time':None,'z':np.nan}
                            stats['intraday_exits'] += 1
                            continue

                    # ENTRY (only if no open position and not entered today)
                    if pos['shares'] == 0 and not entered_today.get(t, False):
                        z = df['z'].iloc[i] if 'z' in df.columns else np.nan
                        vol15 = df['vol15'].iloc[i] if 'vol15' in df.columns else 0
                        median_vol = df['volume'].median() if len(df)>0 else 0
                        vol_ok = (median_vol > 0) and (vol15 >= median_vol * VOLUME_MIN_FACTOR)

                        today_close = daily_closes[t].loc[current_date] if current_date in daily_closes[t].index else np.nan
                        trend_val = daily_trends[t].loc[current_date] if current_date in daily_trends[t].index else np.nan
                        trend_bull = pd.notna(today_close) and pd.notna(trend_val) and (today_close > trend_val)

                        confirm_pass = True
                        if CONFIRM_BARS > 0:
                            confirm_pass = True
                            for j in range(CONFIRM_BARS):
                                idx = i - j
                                if idx < 0 or idx >= len(df) or not np.isfinite(df['z'].iloc[idx]) or df['z'].iloc[idx] > -Z_THRESHOLD:
                                    confirm_pass = False
                                    break

                        if not np.isfinite(z):
                            stats['z_fail'] += 1; continue
                        if z > -Z_THRESHOLD:
                            stats['z_fail'] += 1; continue
                        if not vol_ok:
                            stats['vol_fail'] += 1; continue
                        if not trend_bull:
                            stats['trend_fail'] += 1; continue
                        if not confirm_pass:
                            stats['confirm_fail'] += 1; continue

                        remaining_capacity = max(0.0, (total_value * MAX_GROSS_EXPOSURE) - gross_exposure)
                        if remaining_capacity <= 0:
                            stats['size_fail'] += 1; continue

                        if not np.isfinite(exec_price) or exec_price <= 0:
                            stats['price_fail'] += 1; continue

                        vol = df['volatility'].iloc[i] if 'volatility' in df.columns else VOL_FLOOR
                        atr = df['atr'].iloc[i] if 'atr' in df.columns else ATR_FLOOR
                        if not np.isfinite(vol) or vol <= 0: vol = VOL_FLOOR
                        if not np.isfinite(atr) or atr <= 0: atr = ATR_FLOOR

                        dollar_risk_per_share = max(atr, exec_price * vol, ATR_FLOOR)
                        risk_budget = total_value * RISK_PER_TRADE

                        if dollar_risk_per_share <= 0 or not np.isfinite(dollar_risk_per_share):
                            stats['size_fail'] += 1; continue

                        approx_shares = int(math.floor(risk_budget / dollar_risk_per_share))
                        max_shares_by_fraction = int(math.floor((total_value * MAX_POSITION_FRACTION) / max(exec_price, 1e-6)))
                        approx_shares = max(0, min(approx_shares, max_shares_by_fraction))
                        if approx_shares <= 0:
                            stats['size_fail'] += 1; continue

                        allowed_value = min(approx_shares * exec_price, remaining_capacity)
                        n_shares = int(allowed_value // exec_price)
                        if n_shares <= 0:
                            stats['size_fail'] += 1; continue

                        cost = n_shares * exec_price * (1.0 + TRANSACTION_COST_PCT + SLIPPAGE_PCT)
                        if cost > cash:
                            stats['cash_fail'] += 1; continue

                        if cost > cash * 0.75:
                            stats['size_fail'] += 1; continue

                        # EXECUTE buy -- store z-score inside position
                        cash -= cost
                        positions[t] = {'shares': n_shares, 'entry_price': exec_price, 'entry_time': df['timestamp'].iloc[i+1], 'z': float(z)}
                        entered_today[t] = True
                        stats['entries'] += 1

                # mark-to-market snapshot
                total_value = cash + sum(pos['shares'] * current_prices.get(sym, pos['entry_price'])
                                         for sym,pos in positions.items() if pos['shares'] != 0)
                ts = None
                for df in intraday.values():
                    if df is not None and i+1 < len(df):
                        ts = df['timestamp'].iloc[i+1]; break
                if ts is not None:
                    portfolio_history.append((ts, total_value))

                if cash < INITIAL_CAPITAL * 0.01:
                    print(f"[WARN] Cash very low: {cash:.2f} on {date_str} i={i}. Continuing but check sizing.")

            # End of day: close all positions
            for sym, pos in list(positions.items()):
                if pos['shares'] != 0:
                    df = intraday.get(sym)
                    if df is not None and len(df) > 0:
                        close_price = df['close'].iloc[-1]
                        proceeds = pos['shares'] * close_price * (1.0 - TRANSACTION_COST_PCT - SLIPPAGE_PCT)
                        cash += proceeds
                        trades.append({
                            'symbol': sym,
                            'entry_price': pos['entry_price'],
                            'exit_price': close_price,
                            'entry_time': pos.get('entry_time'),
                            'exit_time': pd.to_datetime(current_date) + pd.Timedelta(hours=16),
                            'shares': pos['shares'],
                            'pnl': (close_price - pos['entry_price']) * pos['shares'],
                            'pnl_pct': (close_price - pos['entry_price']) / (pos['entry_price'] if pos['entry_price']!=0 else 1e-8),
                            'z_score': pos.get('z', np.nan)
                        })
                        stats['eod_closes'] += 1
                    positions[sym] = {'shares':0, 'entry_price':0.0, 'entry_time':None, 'z':np.nan}

            portfolio_history.append((pd.to_datetime(current_date) + pd.Timedelta(hours=16), cash))

        except Exception as e:
            print("Exception on date", date_str, e)
            traceback.print_exc()
            portfolio_history.append((pd.to_datetime(current_date) + pd.Timedelta(hours=16), cash))
            continue

    idx = pd.DatetimeIndex([t for t,_ in portfolio_history])
    vals = [v for _,v in portfolio_history]
    series = pd.Series(vals, index=idx).sort_index().resample('1min').last().ffill().fillna(INITIAL_CAPITAL)
    trades_df = pd.DataFrame(trades)
    # return also daily_closes/daily_trends for reporting
    return series, trades_df, stats, daily_closes, daily_trends

# ------------------------
# Performance helpers and reporting
# ------------------------
def summarize_performance(series, trades_df):
    total_return = series.iloc[-1] / series.iloc[0] - 1
    days = max(1, (series.index[-1].date() - series.index[0].date()).days)
    annualized = (1 + total_return) ** (252/days) - 1
    rolling_max = series.expanding(min_periods=1).max()
    dd = (series - rolling_max) / rolling_max
    max_dd = dd.min()
    rets = series.pct_change().dropna()
    sharpe = (rets.mean() / rets.std()) * math.sqrt(252*MINUTES_PER_DAY) if rets.std() > 0 else 0
    return {'total_return': total_return, 'annualized_return': annualized, 'max_drawdown': max_dd, 'sharpe': sharpe}

# ------------------------
# Run (main)
# ------------------------
if __name__ == "__main__":
    print("🚀 Running (SEQUENTIAL)\n")
    # run backtest
    series, trades_df, stats, daily_closes, daily_trends = run_backtest(TICKERS, START_DATE, END_DATE)

    perf = summarize_performance(series, trades_df)

    # Export trades CSV in MD's exact format
    if not trades_df.empty:
        trades_df['date'] = pd.to_datetime(trades_df['entry_time']).dt.date
        def safe_daily_sma(row):
            try:
                sym = row['symbol']; d = pd.Timestamp(row['date'])
                if sym in daily_trends and d in daily_trends[sym].index:
                    return daily_trends[sym].loc[d]
            except Exception:
                return np.nan
            return np.nan
        trades_df['daily_sma'] = trades_df.apply(safe_daily_sma, axis=1)
        trades_df['stop_loss'] = trades_df['entry_price'] * (1 - STOP_LOSS_PCT)
        trades_df['target'] = trades_df['entry_price'] * (1 + TAKE_PROFIT_PCT)
        # z_score should already be present from run_backtest; if not, fill with NaN
        trades_df['z_score'] = trades_df.get('z_score', np.nan).astype(float)

        final_trades = trades_df[['entry_time','symbol','entry_price','stop_loss','target','pnl','daily_sma','z_score']].copy()
        final_trades.columns = ['TimeOfSignal','Ticker','EntryPrice','StopLoss','TargetPrice','RealizedPnL','DailySMAValue','ZScore']
        out_csv = os.path.join(OUT_DIR, "QuantX_V4.9_Trades_Report.csv")
        final_trades.to_csv(out_csv, index=False)
        print(f"✅ Trade report exported: {out_csv}")
    else:
        final_trades = pd.DataFrame(columns=['TimeOfSignal','Ticker','EntryPrice','StopLoss','TargetPrice','RealizedPnL','DailySMAValue','ZScore'])
        print("ℹ️ No trades executed — empty trade report created.")

    # Diagnostics summary
    print("\n--- Diagnostics Summary ---")
    print(f"entries: {stats.get('entries',0)}")
    print(f"intraday_exits: {stats.get('intraday_exits',0)}")
    print(f"eod_closes: {stats.get('eod_closes',0)}")
    print(f"z_fail: {stats.get('z_fail',0)}")
    print(f"vol_fail: {stats.get('vol_fail',0)}")
    print(f"trend_fail: {stats.get('trend_fail',0)}")
    print(f"confirm_fail: {stats.get('confirm_fail',0)}")
    print(f"size_fail: {stats.get('size_fail',0)}")
    print(f"cash_fail: {stats.get('cash_fail',0)}")
    print(f"price_fail: {stats.get('price_fail',0)}")
    print(f"missing_data: {stats.get('missing_data',0)}")
    print("----------------------------\n")

    print(f"Initial capital: ${series.iloc[0]:,.2f}")
    print(f"Final capital:   ${series.iloc[-1]:,.2f}")
    print(f"Total return:    {perf['total_return']*100:.2f}%")
    print(f"Annualized:      {perf['annualized_return']*100:.2f}%")
    print(f"Max drawdown:    {perf['max_drawdown']*100:.2f}%")
    print(f"Sharpe (ann):    {perf['sharpe']:.2f}")
    print(f"Trades executed: {len(trades_df)}\n")

    if not trades_df.empty:
        win_rate = (trades_df['pnl'] > 0).mean()
        avg_win = trades_df.loc[trades_df['pnl']>0, 'pnl'].mean()
        avg_loss = trades_df.loc[trades_df['pnl']<=0, 'pnl'].mean()
        print(f"Win rate:        {win_rate*100:.2f}%")
        print(f"Avg Win:         {avg_win:.2f},  Avg Loss: {avg_loss:.2f}")

    # Save equity & PnL hist
    try:
        eq_png = os.path.join(OUT_DIR, "QuantX_V4.9_EquityCurve.png")
        plt.figure(figsize=(12,5))
        plt.plot(series.index, series.values, linewidth=1)
        plt.title(f"Equity Curve ({RUN_MODE}): {START_DATE} → {END_DATE}")
        plt.ylabel("Portfolio value ($)")
        plt.grid(True)
        plt.savefig(eq_png, bbox_inches='tight', dpi=150)
        plt.close()
        print(f"✅ Equity curve saved: {eq_png}")

        if not trades_df.empty:
            hist_png = os.path.join(OUT_DIR, "QuantX_V4.9_PnL_Hist.png")
            plt.figure(figsize=(8,4))
            plt.hist(trades_df['pnl'].dropna(), bins=50)
            plt.title("Trade PnL Distribution")
            plt.xlabel("PnL")
            plt.ylabel("Frequency")
            plt.grid(True)
            plt.savefig(hist_png, bbox_inches='tight', dpi=150)
            plt.close()
            print(f"✅ PnL hist saved: {hist_png}")
    except Exception as e:
        print("Failed to save equity/pnl charts:", e)

    # Per-ticker charts (one chart per ticker with all entries/exits marked)
    saved_charts = []
    try:
        traded_tickers = sorted(final_trades['Ticker'].unique()) if not final_trades.empty else []
        # If you want *all* tickers regardless of trades, change to TICKERS
        tickers_to_plot = list(traded_tickers)[:MAX_TICKER_CHARTS]

        for sym in tickers_to_plot:
            sym_trades = final_trades[final_trades['Ticker'] == sym]
            # build daily series for plotting if available, else skip
            if sym not in daily_closes or daily_closes[sym].dropna().empty:
                # skip if no daily data
                continue
            price_series = daily_closes[sym].dropna()
            fig, ax = plt.subplots(figsize=(12,5))
            ax.plot(price_series.index, price_series.values, '-', linewidth=1, label='Daily Close')
            # plot markers for each trade
            for _, r in sym_trades.iterrows():
                try:
                    t_entry = pd.to_datetime(r['TimeOfSignal'])
                except Exception:
                    t_entry = pd.to_datetime(r['TimeOfSignal'], errors='coerce')
                entry_y = r['EntryPrice']
                ax.scatter([t_entry], [entry_y], marker='^', color='green', s=50, label='Entry' if _==sym_trades.index[0] else "")
                # for exit, we don't have explicit exit_time in final_trades columns (we stripped it earlier)
                # find original record in trades_df to get exit_time/exit_price
                orig = trades_df[(trades_df['symbol']==sym) & (pd.to_datetime(trades_df['entry_time'])==t_entry)]
                if not orig.empty:
                    exit_time = orig['exit_time'].iloc[0]
                    exit_price = orig['exit_price'].iloc[0]
                    try:
                        exit_time = pd.to_datetime(exit_time)
                    except Exception:
                        exit_time = pd.to_datetime(exit_time, errors='coerce')
                    ax.scatter([exit_time], [exit_price], marker='v', color='red', s=50, label='Exit' if _==sym_trades.index[0] else "")
            ax.set_title(f"{sym} | All trades ({len(sym_trades)} trades)")
            ax.legend(loc='upper left')
            ax.grid(True)
            fname = os.path.join(OUT_DIR, f"chart_{sym}_all_trades.png")
            fig.savefig(fname, bbox_inches='tight', dpi=150)
            plt.close(fig)
            saved_charts.append(fname)
        print(f"✅ Per-ticker charts saved (up to {MAX_TICKER_CHARTS}) in {OUT_DIR}")
    except Exception as e:
        print("Failed to save per-ticker charts:", e)
        traceback.print_exc()

    # Optional: assemble PDF
    try:
        if GENERATE_PDF:
            pdf_path = os.path.join(OUT_DIR, "QuantX_V4.9_Report.pdf")
            with PdfPages(pdf_path) as pdf:
                # first page: text summary
                fig, ax = plt.subplots(figsize=(11,8.5))
                ax.axis('off')
                txt = f"QuantX V4.9 Report\nMode: {RUN_MODE}\nDate range: {START_DATE} → {END_DATE}\n\n"
                txt += f"Initial capital: ${series.iloc[0]:,.2f}\nFinal capital: ${series.iloc[-1]:,.2f}\nTotal return: {perf['total_return']*100:.2f}%\nAnnualized: {perf['annualized_return']*100:.2f}%\nMax drawdown: {perf['max_drawdown']*100:.2f}%\nSharpe (ann): {perf['sharpe']:.2f}\n\n"
                txt += f"Trades executed: {len(trades_df)}\nEntries: {stats.get('entries',0)}  intraday exits: {stats.get('intraday_exits',0)}  eod closes: {stats.get('eod_closes',0)}\n"
                ax.text(0.01, 0.99, txt, va='top', ha='left', fontsize=10, family='monospace')
                pdf.savefig(fig); plt.close(fig)

                # equity figure
                if os.path.exists(eq_png):
                    fig = plt.figure(figsize=(11,6))
                    img = plt.imread(eq_png)
                    plt.imshow(img); plt.axis('off')
                    pdf.savefig(fig); plt.close(fig)

                # per-ticker charts (append)
                for p in saved_charts:
                    fig = plt.figure(figsize=(11,6))
                    img = plt.imread(p)
                    plt.imshow(img); plt.axis('off')
                    pdf.savefig(fig); plt.close(fig)

                # include small table page (top 20 trades)
                if not final_trades.empty:
                    fig, ax = plt.subplots(figsize=(11,8.5))
                    ax.axis('off')
                    sample_table = final_trades.head(40)
                    table = ax.table(cellText=sample_table.values,
                                     colLabels=sample_table.columns,
                                     cellLoc='center',
                                     loc='center')
                    table.auto_set_font_size(False)
                    table.set_fontsize(8)
                    table.scale(1, 1.5)
                    pdf.savefig(fig); plt.close(fig)

            print(f"✅ Combined PDF report saved: {pdf_path}")
    except Exception as e:
        print("Failed to assemble PDF:", e)

QuantX FINAL Backtest | RUN_MODE=FULL_YEAR
Running 32 tickers: ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'META', 'NVDA', 'JPM', 'BAC', 'GS', 'MS', 'WFC', 'JNJ', 'UNH', 'PFE', 'MRK', 'ABT', 'XOM', 'CVX', 'CAT', 'GE', 'MMM', 'PG', 'KO', 'WMT', 'MCD', 'NKE', 'PEP', 'HD', 'DIS', 'V', 'MA', 'ZTS']
Date range: 2024-01-02 → 2024-12-31

🚀 Running QuantX V4.9 MD Report (SEQUENTIAL)

Processing 20240102
Processing 20240103
Processing 20240104
Processing 20240105
Processing 20240108
Processing 20240109
Processing 20240110
Processing 20240111
Processing 20240112
Processing 20240115
Processing 20240116
Processing 20240117
Processing 20240118
Processing 20240119
Processing 20240122
Processing 20240123
Processing 20240124
Processing 20240125
Processing 20240126
Processing 20240129
Processing 20240130
Processing 20240131
Processing 20240201
Processing 20240202
Processing 20240205
Processing 20240206
Processing 20240207
Processing 20240208
Processing 20240209
Processing 20240212
Processing 20240213
Processing 2